## Week2: TensorFlow and Keras Model Training

This notebook serves as part of the **Computer Intelligence and its Applications in Mechatronics** course at **Amirkabir University of Technology**. In this notebook, we will demonstrate how to create, train, and evaluate neural network models using TensorFlow and Keras. We will cover both the Sequential and Functional API approaches for building models. Additionally, we will implement a custom training loop using TensorFlow's GradientTape.

### Steps Covered:

1. **Importing Libraries**: We import necessary libraries including TensorFlow, Keras, and other utility libraries for data preprocessing and visualization.
2. **Data Preparation**: We generate a synthetic multi-class dataset, split it into training and testing sets, and standardize the features. The labels are converted to categorical format.
3. **Sequential Model Implementation**: We build a neural network using Keras' Sequential API, compile it, and train it on the prepared dataset. The model's performance is evaluated on the test set.
4. **Functional Model Implementation**: We build the same neural network using Keras' Functional API, compile it, and train it similarly. The model's performance is evaluated on the test set.
5. **Custom Training Loop**: We implement a custom training loop using TensorFlow's GradientTape to manually compute gradients and update model weights.


### Importing Libraries

In this section, we import the necessary libraries for our neural network model training and evaluation. These include:

- `numpy` for numerical operations.
- `matplotlib.pyplot` for plotting and visualization.
- `tensorflow` and `keras` for building and training neural network models.
- `sklearn` for data preprocessing and evaluation metrics.
- `IPython.display` for clearing output in Jupyter Notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from IPython.display import clear_output

### Custom Callback for Plotting Losses

In this section, we define a custom callback class `PlotLosses` that inherits from `keras.callbacks.Callback`. This callback will be used to plot the training and validation losses at the end of each epoch. The `on_train_begin` method initializes the variables needed for plotting, and the `on_epoch_end` method updates the plot with the latest loss values. This visualization helps in monitoring the model's performance during training.

In [ ]:
class PlotLosses(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.fig = plt.figure()

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1

        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Losses')
        plt.legend()
        plt.show()

plot_losses = PlotLosses()

### Data Preparation

In this section, we generate a synthetic multi-class dataset using `make_classification` from `sklearn.datasets`. The dataset is then split into training and testing sets. We standardize the features using `StandardScaler` and convert the labels to categorical format using `keras.utils.to_categorical`. This prepares the data for training our neural network models.


In [ ]:
# Generate a multi-class dataset
X, y = make_classification(n_samples=500, n_features=4, n_informative=4, n_redundant=0,
                           n_classes=3, n_clusters_per_class=1, random_state=42)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to categorical (one-hot encoding)
y_train_cat = keras.utils.to_categorical(y_train, num_classes=3)
y_test_cat = keras.utils.to_categorical(y_test, num_classes=3)

## Sequential Model Implementation

In this section, we implement a neural network using Keras' Sequential API. The model consists of an input layer, several hidden layers with ReLU activation, and an output layer with softmax activation for multi-class classification. We compile the model with the Adam optimizer and categorical cross-entropy loss. The model is then trained on the prepared dataset, and its performance is evaluated on the test set.

In [ ]:
### Sequential Model Implementation ###
sequential_model = Sequential([
    Input(shape=(4,)),
    Dense(16, activation='relu'),
    Dense(5, activation='relu'),
    Dense(3, activation='softmax')
])

### Training the Sequential Model

In this section, we compile and train the sequential model defined in the previous cell. We use the Adam optimizer and categorical cross-entropy loss. After training, we evaluate the model's performance on the test set and print the test accuracy.

In [ ]:
# Compile the model
sequential_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = sequential_model.fit(X_train, y_train_cat, epochs=200, batch_size=16,
                                validation_data=(X_test, y_test_cat))

# Evaluate the model
test_loss, test_acc = sequential_model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {test_acc:.2f}")


## Functional Model Implementation

In this section, we implement a neural network using Keras' Functional API. The model consists of an input layer, several hidden layers with ReLU activation, and an output layer with softmax activation for multi-class classification. We compile the model with the Adam optimizer and categorical cross-entropy loss. The model is then trained on the prepared dataset, and its performance is evaluated on the test set.


In [ ]:
### Functional Model Implementation ###
inputs = Input(shape=(4,))
x = Dense(16, activation='relu')(inputs)
x = Dense(50, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)
functional_model = Model(inputs, outputs)

In [ ]:
# Compile and train
functional_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
functional_model.fit(X_train, y_train_cat, epochs=200, batch_size=16,
                                validation_data=(X_test, y_test_cat))

# Evaluate the model
test_loss, test_acc = functional_model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {test_acc:.2f}")


## Custom Training Loop

In this section, we implement a custom training loop using TensorFlow's `GradientTape`. This approach allows us to manually compute gradients and update model weights. We use the Adam optimizer and categorical cross-entropy loss. The training loop runs for a specified number of epochs, and the loss is printed every 10 epochs to monitor the training progress.


In [ ]:
optimizer = Adam(learning_rate=0.01)

# Training loop
for epoch in range(200):
    with tf.GradientTape() as tape:
        predictions = sequential_model(X_train, training=True)
        loss = tf.keras.losses.categorical_crossentropy(y_train_cat, predictions)
        loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss, sequential_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, sequential_model.trainable_variables))
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")